In [5]:
import numpy as np 
import matplotlib.pyplot as plt



error = 0 # 1 - there was an error in the programm, 0 - OK
N = 5 # Number of components
#Langmuir Isotherm
SingleComponentCapacity = np.array([10,5.808,5.985,5,5]) #Langmuir Isotherm capacity of every component
AffinityCoefficient = np.array([0.01476,0.01476,0.0098,0.001,0.005]) #Langmuir Affinity Coefficient of every component
fractionGas = np.array([0.1,0.1,0.2,0.2,0.4]) #Gas fraction of every component

#Initialization
fastiastGraphConcentration = np.zeros(N)
fastiastGraphFraction = np.zeros(N)
fastiastPressure = 0
adsorbedFraction = np.zeros(N)
adsorbedConcentration = np.zeros(N)

#Checking..
if (len(fractionGas) < N or len(SingleComponentCapacity) < N or len(AffinityCoefficient) < N):
    print("You have the incorrect number of components")
    error = 1
if np.sum(fractionGas) < 1:
    error = 1
    print("The molar fractions sum is less then 1")

###Calculation###
for P_gas in np.linspace(0.1,100,20):
    kappa_old = np.zeros(N)
    delta_kappa = np.ones(N)
    kappa = np.zeros(N)
    CmuT = 0


    #Step 2. Initial guess for the pure hypothetical pressures
    partialPressureComponents = fractionGas*P_gas
    for k in range(N):
        CmuT += SingleComponentCapacity[k]*AffinityCoefficient[k]*partialPressureComponents[k]

    for k in range(N):
        kappa[k] = CmuT/(SingleComponentCapacity[k])

    i = 0
    while np.any((delta_kappa) > 1e-4):
        f = np.zeros(N)
        fDerivative = np.zeros(N)
        g = np.zeros(N)
        sigma = np.zeros(N)
        phi = np.zeros((N,N))
        phi = np.matrix(phi)

        for k in range(N):
            f[k] = SingleComponentCapacity[k]*(np.log(1+kappa[k]))
            fDerivative[k] = SingleComponentCapacity[k]*(1/(1+kappa[k]))

        for k in range(N-1):
            g[k] = f[k] - f[k+1]
        for k in range(N):
            g[N-1] += AffinityCoefficient[k]*partialPressureComponents[k]/kappa[k]
        g[N-1] = g[N-1] - 1 


        for k in range(N-1):
            phi[k,k] = fDerivative[k]
            phi[k,k+1] = -fDerivative[k+1]
        for k in range(0,N):
            phi[N-1,k] = - (AffinityCoefficient[k]*partialPressureComponents[k]/(kappa[k]**2))

        sigma = np.linalg.solve(phi, g)
        kappa_old = kappa
        kappa = kappa_old - sigma

        delta_kappa = np.abs(kappa-kappa_old)


        i += 1
        if i > 20:
            print("No convergence")
            error = 1
            break
        if np.any(kappa < 0):
            print("No convergence")
            error = 1
            break
        adsorbedFraction = partialPressureComponents*AffinityCoefficient/kappa
        adsorbedConcentrationPure = SingleComponentCapacity*(kappa
                                      /(1+kappa))  

        C_total = 0
        for k in range(0,N):
            C_total += 1 / (adsorbedFraction[k]/adsorbedConcentrationPure[k])
        adsorbedConcentration =  C_total*adsorbedFraction   


    fastiastGraphConcentration=np.vstack((fastiastGraphConcentration, adsorbedConcentration))
    fastiastGraphFraction=np.vstack((fastiastGraphFraction, adsorbedFraction))
    fastiastPressure=np.vstack((fastiastPressure, P_gas))
if error == 0:
    print("Finished")
    print(f"C total is {C_total} at final pressure. Below are the adsorbed amounts of components")
###Result###
print(fastiastGraphConcentration)

Finished
C total is 172.62686633685223 at final pressure. Below are the adsorbed amounts of components
[[0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [9.30669076e-02 5.40442775e-02 7.39540679e-02 6.30389434e-03
  6.30389434e-02]
 [4.83848147e+00 2.78559893e+00 3.81413008e+00 3.23840520e-01
  3.23840520e+00]
 [9.31392690e+00 5.31730721e+00 7.28496083e+00 6.16153385e-01
  6.16153385e+00]
 [1.35465772e+01 7.67055830e+00 1.05151457e+01 8.86009012e-01
  8.86009012e+00]
 [1.75600064e+01 9.86381104e+00 1.35294611e+01 1.13579041e+00
  1.13579041e+01]
 [2.13747740e+01 1.19130793e+01 1.63494093e+01 1.36756096e+00
  1.36756096e+01]
 [2.50088815e+01 1.38323153e+01 1.89937306e+01 1.58311508e+00
  1.58311508e+01]
 [2.84781748e+01 1.56337365e+01 2.14788414e+01 1.78402098e+00
  1.78402098e+01]
 [3.17966523e+01 1.73280823e+01 2.38191769e+01 1.97165454e+00
  1.97165454e+01]
 [3.49767298e+01 1.89248306e+01 2.60274814e+01 2.14722766e+00
  2.14722766e+01]
 [3.80294599e+01 